In [ ]:
import cv2
import CV_utils as cvu
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
%matplotlib inline

## READ FRAMES

In [ ]:
l_b_frames, l_bgs_frames, l_c_frames, l_gs_frames = cvu.read_frames('Video/representative/Lightness/Video attached to the ICMI 2017 Paper - Lightness.mp4')
f_b_frames, f_bgs_frames, f_c_frames, f_gs_frames = cvu.read_frames('Video/representative/Fragility/Video attached to the ICMI 2017 Paper - Fragility.mp4')

## GRADIENT DIFFERENCE

### SHARED FUNCTIONS

In [ ]:
# projection of the gradient along the most significant (fundamental) direction
def compute_proj(g_x, g_y):
    
    proj_counter = np.zeros((cvu.len_ref_angles))
    
    for h in range(cvu.len_ref_angles//2):

        # project
        proj_counter[h] = np.dot([g_x, g_y], cvu.ref_angles[h])

        # if negative it's the origin-symmetric angle
        if proj_counter[h] < 0:
            proj_counter[h+(cvu.len_ref_angles//2)] -= proj_counter[h]
            proj_counter[h] = 0

    #to zero all the projections but the maximum one -- clustering    
    max_index = np.argmax(proj_counter)
    proj_counter[[idx for idx in range(cvu.len_ref_angles) if idx!=max_index]] = 0
    
    return proj_counter

In [ ]:
# compute the gradient for each patch with Sobel
# compute the mean of such gradients in both the directions
# project such mean gradient

def compute_grad_v2(patch_mat, size=3):
    
    proj_mat = np.zeros((patch_mat.shape[0], patch_mat.shape[1], cvu.len_ref_angles))
    
    for i, patch_line in enumerate(patch_mat):
        for j, patch in enumerate(patch_line):
            
            # gradient computation of a patch
            sobelx64f = cv2.Sobel(patch, cv2.CV_64F,1,0,ksize=size)
            abs_sobel64f = np.absolute(sobelx64f)
            grad_x = np.uint8(abs_sobel64f)

            sobely64f = cv2.Sobel(patch, cv2.CV_64F,0,1,ksize=size)
            abs_sobel64f = np.absolute(sobely64f)
            grad_y = np.uint8(abs_sobel64f)
            
            # projection of the mean gradient
            proj_mat[i,j,:] = compute_proj(np.mean(grad_x), np.mean(grad_y))
    
    return proj_mat

In [ ]:
# pooling of neighbour gradient patches
def zoom_out_v2(grads, n_patch): # n_patch tuple (num_patch_y, num_patch_x)
    
    if n_patch[0]%2 == 0 or n_patch[1]%2 == 0:
        raise Exception("n_patch must be a touple of odd numbers")
    
    frames_pools = []
    tmp_y = n_patch[0]//2
    tmp_x = n_patch[1]//2
    
    len_x = grads[0].shape[1]//tmp_x
    len_y = grads[0].shape[0]//tmp_y
    
    for frame_grad in grads:
        
        pools = np.zeros((n_patch[0], n_patch[1], frame_grad.shape[2]))
        
        for x in range(tmp_x):
            for y in range(tmp_y):
                
                pools[x, y, :] = sum(sum(frame_grad[x*len_x:(x+1)*len_x, y*len_y:(y+1)*len_y]))
                
                if x != tmp_x-1:
                    
                    pools[x, y, :] = sum(sum(frame_grad[int((x+0.5)*len_x):int((x+1.5)*len_x), y*len_y:(y+1)*len_y]))
                    
                    if y != tmp_y-1:
                        
                        #pools[x, y, :] = sum(sum(frame_grad[int((x+0.5)*len_x):int((x+1.5)*len_x),int((y+0.5)*len_y):int((y+1.5)*len_y)]))
                        pools[x, y, :] = sum(sum(frame_grad[x*len_x:(x+1)*len_x, int((y+0.5)*len_y):int((y+1.5)*len_y)]))
                
                elif y != tmp_y-1:
                    
                    pools[x, y, :] = sum(sum(frame_grad[x*len_x:(x+1)*len_x, int((y+0.5)*len_y):int((y+1.5)*len_y)]))         
            
        # TODO threshold 80%?
        
        frames_pools.append(pools.reshape(-1, pools.shape[2]))
        
    return frames_pools

In [ ]:
# function to divide an image in patches
def extract_patches(img, patch_size, overlap):
    
    img_shape = img.shape
    
    # compare img size with patch size
    if img_shape[0] < patch_size or img_shape[1] < patch_size:
        raise Exception("Patch size too big. Image shape {}, patch size ({}, {})".format(img_shape, patch_size, patch_size))
    
    # how many patches can fit the img height
    shape_x = ((img_shape[0]-patch_size)//(patch_size-overlap))+1
    
    # if img height is not a multiple of patch_size the last one will be adjusted in some way...
    if (img_shape[0]-patch_size)%(patch_size-overlap) != 0:     
        shape_x += 1
     
    # how many patches can fit the img length    
    shape_y = ((img_shape[1]-patch_size)//(patch_size-overlap))+1
    
    # if img length is not a multiple of patch_size the last one will be adjusted in some way...
    if (img_shape[1]-patch_size)%(patch_size-overlap) != 0:     
        shape_y += 1
    
    # pixels in a patch, the first two values are for patch indexing and the latter ones are the actual img portion:
    # [patch_over_height * patch_over_length * pixel_height * pixel_length]
    patches = np.zeros((shape_x, shape_y, patch_size, patch_size))
    
    h = 0
                        
    for i in range(shape_x):
        l = 0

        for j in range(shape_y):            
            # if it's the last patch along the height...
            if i == shape_x-1:
                # ... and the last one along the length
                if j == shape_y-1:
                    tmp = np.vstack((img[h:, l:], np.zeros((patch_size-(img.shape[0]-h), img.shape[1]-l))))
                    tmp = np.hstack((tmp, np.zeros((patch_size, patch_size-(img.shape[1]-l)))))
                    patches[i, j, :, :] = tmp

                else:
                    patches[i, j, :, :] = np.vstack((img[h:, l:l+patch_size], np.zeros((patch_size-(img.shape[0]-h),patch_size))))

                    
            # if it's just the last patch along the length        
            elif j == shape_y-1:
                patches[i, j, :, :] = np.hstack((img[h:h+patch_size, l:], np.zeros((patch_size, patch_size-(img.shape[1]-l)))))
            
            else:
                patches[i, j, :, :] = img[h:h+patch_size, l:l+patch_size]
            
            l += patch_size-overlap
        h += patch_size-overlap            
    return patches

In [ ]:
# function to compute the difference of the gradient value
#in each fundamental direction in each pool between two consecutive frames

#Moreover it calls the functions to compute the gradients
#and the pools
def compute_diff(frames, size, overlap, n_skip, pools):
    
    grads = []
    diff = []
    
    s = 0
        
    print("\tcalculating the gradient of frames...")
    
    for frame in frames:
        
        if s != 0 and s<= skip:
            s += 1
            continue
            
        patches = extract_patches(frame, size, overlap)
        
        grads.append(compute_grad_v2(patches))
        
        s = 0
    
    print("\tpooling the frames...")
    
    frames_pools = zoom_out_v2(grads, pools)
    
    print("\tcalculating the gradients variation...")
    
    #print(frames_pools)
    
    for i, pools in enumerate(frames_pools[:-1]):
        
        next_pools = frames_pools[i+1]
        diff.append(next_pools-pools)
        
    return diff

### FOURIER

In [ ]:
def fourier_dataset_creation(video_frames_list, size=40, overlap=1, n_skip=0, pools=(5,3), verbose=False):
    
    angles_transformed_list = []
    
    for i, video_frames in enumerate(video_frames_list):
        if verbose:
            tot = len(video_frames_list)
            print('\nWorking on video {}/{}'.format(i+1, tot))
            print('\t.... Computing the gradient difference frame by frame')
            start = time.time()

        diff = compute_diff(video_frames, size, overlap, n_skip, pools)
        
        if verbose:
            print("\t.... complete. Time spent %s" % (time.time()-start))
            
        # n_pools * n_angles * n_frames-1 = n_diff
        angles_signal = np.zeros((diff[0].shape[0], diff[0].shape[1], len(diff)))
        
        for j, d in enumerate(diff): angles_signal[:,:,j] = d
        
        angles_transformed = np.zeros((angles_signal.shape[0],angles_signal.shape[1], int(np.ceil((angles_signal.shape[2]+1)/2)*2)))
        
        if verbose:
            start = time.time()
            print('\tComputing the Fourier serie...')
            
        for h, pool_angles in enumerate(angles_signal):
            for j, angle_signal in enumerate(pool_angles):
                # call the fourier transformation on angle_signal
                coeff = np.fft.rfft(angle_signal)
                angles_transformed[h, j, :] = np.hstack((coeff.real, coeff.imag))
                
            
        if verbose: print('\t.... complete. Time spent: %s'%(time.time()-start))
        
        #flattening of the transformation matrix
        angles_transformed_list.append(angles_transformed.reshape(-1, angles_transformed.shape[2]).reshape(-1))
        
    return np.asmatrix(angles_transformed_list)

In [ ]:
# function to load the data and create the dataset using fourier
def load_gradient(path_to_read_binary, path_to_read_density, path_to_write, width, verbose=False):

    indexes_deleted = []

    # Batch video loading (10*15)
    for j in range(1,16):

        if verbose: print("\nExtracting data for the videos from {} to {}".format((j-1)*10+1, j*10))
        
        video_list = cvu.binary_read(path_to_read_binary, j)
        
        pd_density_frames = pd.read_pickle(path_to_read_density+str(j)+'.pkl')
        
        X, indexes = cvu.normalize_preprocessing(video_list, pd_density_frames, width=width, verbose=verbose)
        
        # invoke the creation of the dataset by fourier
        X = fourier_dataset_creation(X, verbose=verbose)
        
        bDel_indexes = [x+(j-1)*10 for x in indexes]
        indexes_deleted += bDel_indexes
        
        df = pd.DataFrame(X)
        df.to_csv(path_to_write+str(j)+".csv", index=False)
        
    return indexes_deleted

In [ ]:
indexes_deleted = load_gradient('Video/binary_video/','Video/densityFrame_pkl/videos_density_', 'datasets/fourier_gradient_partial/fourier_gradient_dataset_', 650, verbose=True)

pd.DataFrame(indexes_deleted).to_csv('datasets/fourier_gradient_partial/fourier_gradient_indexes_deleted.csv', index=False)

In [ ]:
cvu.merge_dataset('datasets/fourier_gradient_partial/fourier_gradient_dataset_', 'datasets/fourier_gradient_dataset.csv', verbose=True)

### AVG OF SQUARES

In [ ]:
# Given a list of matrices of difference, it returns the mean of the squares of such element, grouped by batch_size
def avg_squared_pools_channels(diff_list, batch_size=0, n_overlap=0, n_skip=0):
    
    if batch_size == 0 : batch_size=len(diff_list)
    
    batch_list = []
    batch = np.zeros(diff_list[0].shape)
    overlap_diff = np.zeros(diff_list[0].shape)
    skip = 0 # consecutive frames skip counter
    i = 0    # seen frames counter (used for the overlapping purpose)
    
    for diff in diff_list:
        
        # skip consecutive frames, pick one every n_skip
        if skip != 0 and skip <= n_skip:
            skip += 1
            continue
        else:
            skip = 0
            
        diff = np.square(diff)
            
        if i >= batch_size-n_overlap:           
            overlap_diff += diff
            
        batch += diff
        
        i += 1
        
        # if it has already seen batch_size imgs
        if i%batch_size == 0:
            
            batch_list.append(batch)
            batch = n_overlap # start from the stored frames
            
            i = n_overlap
            overlap_frames = np.zeros(diff.shape)
        
        skip += 1
        
    return sum(batch_list)/len(batch_list)

In [ ]:
def avg_squared_gradient_dataset_creation(video_frames_list, size=40, overlap=1, n_skip=0, pools=(5,3), verbose=False):
    
    pools_channels_in_time_list = []
    
    if verbose: tot = len(video_frames_list)
       
    for i, video_frames in enumerate(video_frames_list):
                
        if verbose:
            print('\nWorking on video {}/{}'.format(i+1, tot))
            print('\t.... Computing the gradient difference frame by frame')
            start = time.time()

        diff = compute_diff(video_frames, size, overlap, n_skip, pools)
    
        if verbose:
            print("\t.... Complete. Time spent %s" % (time.time()-start))
            print("\t.... Computing avg squared")
            start = time.time()
    
        n_frames = len(diff)
    
        #TODO add a for that perform different approaches like in density
        # if verbose print approach number
        pools_channels_in_time = np.zeros((diff[0].shape))
            
        pools_channels_in_time = avg_squared_pools_channels(diff)
        
        if verbose: print("\t.... Complete. Time spent %s" % (time.time()-start))
    
        pools_channels_in_time_list.append(pools_channels_in_time.reshape(-1))
    
    return np.asmatrix(pools_channels_in_time_list)

In [ ]:
# function to load the data and create the dataset using the avg of the squares
def load_avgSquared_gradient(path_to_read_binary, path_to_read_density, path_to_write, width, 
                                        size=40, n_skip=3, pool_size=(5,3), verbose = False):

    indexes_deleted = []

    # Batch video loading (10*15)
    for j in range(1,16):

        if verbose:
            print("\nExtracting data for the videos from {} to {}".format((j-1)*10+1, j*10))
        
        video_frames_list = cvu.binary_read(path_to_read_binary, j, verbose=verbose)
        
        pd_density_frames = pd.read_pickle(path_to_read_density+str(j)+'.pkl')
        
        X, _ = cvu.normalize_preprocessing(video_frames_list, pd_density_frames, width, n_frames=None, verbose=verbose)
        
        X = avg_squared_gradient_dataset_creation(X, size, n_skip, pool_size, verbose=verbose)
        
        df = pd.DataFrame(X)
        
        df.to_csv(path_to_write+str(j)+".csv", index=False)

In [ ]:
load_avgSquared_gradient('Video/binary_video/', 'Video/densityFrame_pkl/videos_density_', 'datasets/avg_gradient_partial/avg_square_grad_', 650, verbose=True)

In [ ]:
cvu.merge_dataset('datasets/avg_gradient_partial/avg_square_grad_', 'datasets/avg_square_grad.csv', verbose=True)

## LOADING

### FOURIER TESTING

In [ ]:
X = pd.read_csv('datasets/fourier_gradient_dataset.csv').as_matrix()
iDel = pd.read_csv('datasets/fourier_gradient_partial/fourier_gradient_indexes_deleted.csv', index_col=0).values.flatten()
y = cvu.load_marks()[1].values
y = np.delete(y, iDel, axis=0)

In [ ]:
#reordering

X_new = np.zeros(X.shape)

for mul in range(0, 15*8):
    for base in range(250):
        X_new[:, base+mul*250] = X[:, base+mul*500]
        X_new[:, base+mul*250+1] = X[:, base+250+mul*500]

### AVG TESTING

In [ ]:
X = pd.read_csv('datasets/avg_square_grad.csv').as_matrix()
y = cvu.load_marks()[1].values

## TESTING

In [ ]:
y_l = y[:,0]
y_f = y[:,1]

In [ ]:
X_tr, Y_l_tr, Y_f_tr, X_ts, Y_l_ts, Y_f_ts = cvu.random_sampling(X, y_l, y_f)
#X_tr, X_ts, Y_l_tr, Y_f_tr, Y_l_ts, Y_f_ts = cvu.split_tr_ts(X, y_l, y_f)

RLS

In [ ]:
alphas = np.arange(0.01, 7, 0.01)

In [ ]:
# LIGHTNESS
cvu.print_results('RLS',cvu.rlsCV_regression(alphas, X_tr, X_ts, Y_l_tr, Y_l_ts))

In [ ]:
# FRAGILITY
cvu.print_results('RLS', cvu.rlsCV_regression(alphas, X_tr, X_ts, Y_f_tr, Y_f_ts))

LASSO

In [ ]:
alphas = np.arange(0.001, 1, 0.007)

In [ ]:
#LIGHTNESS
cvu.print_results('LASSO', cvu.lassoCV_regression(alphas, X_tr, X_ts, Y_l_tr, Y_l_ts))

In [ ]:
#FRAGILITY
cvu.print_results('LASSO', cvu.lassoCV_regression(alphas, X_tr, X_ts, Y_f_tr, Y_f_ts))

KERNEL

In [ ]:
alphas = {'alpha': list(np.arange(0.001, 1, 0.007))}
kernel = 'rbf'

In [ ]:
#LIGHTNESS
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, Y_l_tr, Y_l_ts, kernel=kernel))

In [ ]:
#FRAGILITY
cvu.print_results('RIDGE KERNEL', cvu.ridgeKernelCV_regression(alphas, X_tr, X_ts, Y_f_tr, Y_f_ts, kernel=kernel))

SVM

In [ ]:
c = {'C':list(np.arange(1, 100, 10))}

In [ ]:
#LIGHTNESS
cvu.print_results('SVM', cvu.svmCV_regression(c, X_tr, X_ts, Y_l_tr, Y_l_ts))

In [ ]:
#FRAGILITY
cvu.print_results('SVM', cvu.svmCV_regression(c, X_tr, X_ts, Y_f_tr, Y_f_ts))

RF

In [ ]:
ris = cvu.tree_regression(X_tr, X_ts, Y_f_tr, Y_f_ts)

In [ ]:
print(ris[0], ris[1])
print(ris[2])

GROUP LASSO

In [ ]:
alphas = np.arange(0.001, 1, 0.007)

In [ ]:
groups = []

for i in range(30000):
    groups.append([i*2, i*2+1])

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Lightness
coef, alpha = cvu.group_lassoCV(X_tr, Y_l_tr, alphas, groups, max_iter=cvu.MAX_ITER, rtol=1e-6)
err = sum(abs(Y_l_ts-np.dot(X_ts, coef)))/len(Y_l_ts)
var = np.var(abs(Y_l_ts-np.dot(X_ts, coef)))
print(err)
print(var)
print(coef)

In [ ]:
#Fragility
coef, alpha = cvu.group_lassoCV(X_tr, Y_f_tr, alphas, groups, max_iter=cvu.MAX_ITER, rtol=1e-6)
err = sum(abs(Y_f_ts-np.dot(X_ts, coef)))
var = np.var(abs(Y_f_ts-np.dot(X_ts, coef)))
print(err)
print(var)
print(coef)